<a href="https://colab.research.google.com/github/rahulguptagzb09/Bidirectional_Tool_Natural_Text_Python/blob/main/bidirectional_mapping_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# !pip install openai

In [9]:
import sys
import collections
import platform

print(sys.version_info)
# Check if running Python version > 3.9
if sys.version_info >= (3, 9):
  if not hasattr(collections, 'MutableMapping'):
      import collections.abc
      collections.MutableMapping = collections.abc.MutableMapping

sys.version_info(major=3, minor=11, micro=12, releaselevel='final', serial=0)


In [10]:
import ipywidgets as widgets
from IPython.display import display
import openai

from google.colab import userdata
userdata.get('openai_api_key')

openai.api_key = userdata.get('openai_api_key')  # Replace with your actual API key

In [11]:
# Define Widgets
natural_input = widgets.Textarea(
    placeholder='Enter logic in natural language...',
    description='Natural Language:',
    layout=widgets.Layout(width='100%', height='120px')
)

structured_output = widgets.Textarea(
    placeholder='Structured text (intermediate representation)...',
    description='Structured Text:',
    layout=widgets.Layout(width='100%', height='120px')
)

code_output = widgets.Textarea(
    placeholder='Python code generated from structured text...',
    description='Python Code:',
    layout=widgets.Layout(width='100%', height='120px')
)

convert_button = widgets.Button(description="Convert")

In [12]:
# OpenAI Translation Functions
def call_chatgpt(prompt):
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

def natural_to_structured(nl_text):
    prompt = f"""You are a code assistant.
Convert the following natural language into unambiguous structured pseudocode instructions.
Use a consistent and readable format that can be reliably mapped to Python code.
Give only structured text or pseudocode in output and nothing else.
Natural language:
{nl_text}
Structured logic:"""
    return call_chatgpt(prompt)

def structured_to_code(structured_text):
    prompt = f"""You are a code assistant.
Convert the following structured pseudocode into valid Python code.
Give only Python code in output and nothing else.
Structured logic:
{structured_text}

Python code:"""
    return call_chatgpt(prompt)

def code_to_structured(code_text):
    prompt = f"""You are a code assistant.
Explain the following Python code in structured pseudocode format.
Maintain clarity and consistency with examples provided.
Give only structured text or pseudocode in output and nothing else.
Python code:
{code_text}

Structured logic:"""
    return call_chatgpt(prompt)

In [13]:
# Event Handlers
def on_convert_clicked(b):
    structured = natural_to_structured(natural_input.value)
    structured_output.value = structured

    code = structured_to_code(structured)
    code_output.value = code

def on_code_edit(change):
    if change['type'] == 'change' and change['name'] == 'value':
        structured = code_to_structured(change['new'])
        structured_output.value = structured

convert_button.on_click(on_convert_clicked)
code_output.observe(on_code_edit)

In [14]:
# Display UI
display(natural_input, structured_output, code_output, convert_button)

Textarea(value='', description='Natural Language:', layout=Layout(height='120px', width='100%'), placeholder='…

Textarea(value='', description='Structured Text:', layout=Layout(height='120px', width='100%'), placeholder='S…

Textarea(value='', description='Python Code:', layout=Layout(height='120px', width='100%'), placeholder='Pytho…

Button(description='Convert', style=ButtonStyle())